In [ ]:
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import gensim
from gensim import corpora, models, similarities

In [ ]:
import re
from nltk.stem import SnowballStemmer
df = pd.read_csv('Training_Full_V1.1.csv', error_bad_lines=False)
def clean_text(text):

    ## Remove puncuation
    text = text.translate(string.punctuation)

    ## Convert words to lower case and split them
    text = text.lower().split()

    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]

    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"<|;", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text
# apply the above function to df['text']
df['sentence'] = df['sentence'].map(lambda x: clean_text(x))


In [ ]:
tok_corp= [nltk.word_tokenize(sent) for sent in df['sentence']]
model = gensim.models.Word2Vec(tok_corp, min_count=1, size = 100)

In [ ]:
train = []
max_len = 0
for i in range(len(df['sentence'])):
    if max_len<len(df['sentence'][i].split(" ")):
        max_len = len(df['sentence'][i].split(" "))
max_len

184

In [ ]:
train = []
blank = [0]*100
for i in range(len(df['sentence'])):
    sent_prob = []
    sent = df['sentence'][i].split(" ")
    if len(sent) >50:
        for k in range(50):
            prob=[]
            word_prob = model.wv.most_similar(sent[k],topn=100)
            for j in range(len(word_prob)):
                prob.append(word_prob[j][1])
            sent_prob.append(prob)
    else:
        for k in range(len(sent)):
            prob=[]
            word_prob = model.wv.most_similar(sent[k],topn=100)
            for j in range(len(word_prob)):
                prob.append(word_prob[j][1])
            sent_prob.append(prob)

        for k in range(50-len(sent)):
            sent_prob.append(blank)
    train.append(sent_prob)
    if i == 3440:
        break

/home/ksrao/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
labels = []
for l in df['label'][0:3441]:
    labels.append(l)

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
X,y = shuffle(train,labels,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X,labels, test_size=0.3)

In [ ]:
def flat_func(data):
    data1 = []
    for i in range(len(data)):
        data1.append(data[i].flatten())
    return data1

In [ ]:
def onehotcode(labels):
    y_true = []
    for i in range(len(labels)):
        list1 = np.zeros(2)
        list1[labels[i]] = 1
        y_true.append(list1)
    return y_true

In [ ]:
x_train = flat_func(np.array(X_train))
x_test = flat_func(np.array(X_test))
y_train_label = onehotcode(y_train)
y_test_label  = onehotcode(y_test)


In [ ]:
len(x_train)

In [ ]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [ ]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [ ]:
def conv2d(x,W):
    # x----> [batch,H,W,Channels]
    # W----> [filter H, filter W , Channels IN, Channels OUT]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [ ]:
def max_pool_2by2(x):
    # x---> [batch,H,W,Channels]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [ ]:
# Convolutional Layer
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [ ]:
# Normal (fully connected)
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+b

In [ ]:
x = tf.placeholder(tf.float32,shape=[None,5000])

In [ ]:
y_true = tf.placeholder(tf.float32,shape=[None,2])

In [ ]:
x_image = tf.reshape(x,[-1,50,100,1])

In [ ]:
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [ ]:
convo_2_flat = tf.reshape(convo_1_pooling,[-1,25*50*32])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [ ]:
#Dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [ ]:
y_pred = normal_full_layer(full_one_dropout,2)

In [ ]:
#Lost Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
saver = tf.train.Saver()

In [ ]:
steps = 599
with tf.Session() as sess:
    sess.run(init)
    j=0
    k=0
    for i in range(1,steps):
        j = j+8
        batch_x,batch_y = x_train[k*8:j],y_train_label[k*8:j]
        k = k+1
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        #3conv_result= (sess.run(convo_1,feed_dict={x:batch_x}))
        #conv_pooling_result= (sess.run(convo_1_pooling,feed_dict={x:batch_x}))
        if i%299==0:
            j=0
            k=0

        if i%100 == 0:
            print("ON STEP:{}".format(i),end=' ')
            print("------------>",end=' ')
            print("ACCURACY: ",end=' ')
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:x_test,y_true:y_test_label,hold_prob:1.0}))
    saver.save(sess,'digit_model/weights_v2.ckpt')
    #result = sess.run(y_pred,feed_dict={x:[data],hold_prob:1.0})